# Basic Search

The CDA provides a custom python tool for searching CDA data. [`Q`](usage/#q) (short for Query) offers several ways to search and filter data, and several input modes:

---
- **<a href="../../QuickStart/usage/#q">Q.()</a>** builds a query that can be used by `run()` or `count()`
- **<a href="../../QuickStart/usage/#qrun">Q.run()</a>** returns data for the specified search 
- **<a href="../../QuickStart/usage/#qcount">Q.count()</a>** returns summary information (counts) data that fit the specified search
- **<a href="../../QuickStart/usage/#columns">columns()</a>** returns entity field names
- **<a href="../../QuickStart/usage/#unique_terms">unique_terms()</a>** returns entity field contents

---
                                                                    
Before we do any work, we need to import these functions from cdapython. We're also importing functions from several other packages to make viewing and manipulating tables easier. The `opt.` settings are pre-configuring how itables should display our tables, with scrolling and paging enabled.
Finally, we're telling cdapython to report it's version so we can be sure we're using the one we mean to:

In [1]:
from cdapython import Q, columns, unique_terms, query
import numpy as np
import pandas as pd
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=True)
import itables.options as opt
opt.maxBytes=0
opt.scrollX="200px"
opt.scrollCollapse=True
opt.paging=True
opt.maxColumns=0
print(Q.get_version())

<IPython.core.display.Javascript object>

2022.12.21

<div class="cdanote" style="background-color:#b3e5d5;color:black;padding:20px;">
    
CDA data comes from three sources:
<ul>
<li><b>The <a href="https://proteomic.datacommons.cancer.gov/pdc/"> Proteomic Data Commons</a> (PDC)</b></li>
<li><b>The <a href="https://gdc.cancer.gov/">Genomic Data Commons</a> (GDC)</b></li>
<li><b>The <a href="https://datacommons.cancer.gov/repository/imaging-data-commons">Imaging Data Commons</a> (IDC)</b></li>
</ul> 
    
The CDA makes this data searchable in four main endpoints:

<ul>
<li><b>subject:</b> A patient entity captures the study-independent metadata for research subjects. Human research subjects are usually not traceable to a particular person to protect the subjects privacy.</li>
<li><b>researchsubject:</b> A research subject is the entity of interest in a specific research study or project, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subjects privacy. This entity plays the role of the case_id in existing data. A subject who participates in 3 studies will have 3 researchsubject IDs.</li>
<li><b>specimen:</b> Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</li>
<li><b>mutation:</b> Molecular data about specific mutations, currently limited to the TCGA-READ project from GDC.</li>
</ul>
one endpoint that can be added to the subject, researchsubject, or specimen to get the relevant files:    
<ul>
<li><b>file:</b> A unit of data about subjects, researchsubjects, specimens, or their associated information.</li>
</ul>
and two endpoints that offer deeper information about data in the researchsubject endpoint:
<ul>
<li><b>diagnosis:</b> A collection of characteristics that describe an abnormal condition of the body as assessed at a point in time. May be used to capture information about neoplastic and non-neoplastic conditions.</li>
<li><b>treatment:</b> Represent medication administration or other treatment types.</li>
</ul>
</div>


If you are looking to build a cohort of distinct individuals who meet some criteria, search by `subject`. If you want to build a cohort, but are particularly interested in studies rather than the participates per se, search by `researchsubject`. If you are looking for biosamples that can be ordered or a specific format of information (for e.g. histological slides) start with `specimen`. If you are primarily looking for files you can reuse for your own analysis, add `.file` to your call.

In the subject, researchsubject, or specimen tables, all of the rows will have one or more files associated with them that can be directly found by chaining, as in `specimen.files`. Diagnosis and treatment do not have files directly associated with them, so a query statemet of `diagnosis.files` or `treatment.file` will not work. The mutation table does have files associated with it, but currently they cannot be accessed with `mutation.file`. Look for this feature in a later release.

Any metadata field can be searched from any endpoint, the only difference between search types is what type of data is returned by default. This means that you can think of the CDA as a really, really enormous spreadsheet full of data. To search this enormous spreadsheet, you'd want select columns, and then filter rows.



## Basic search with endpoints

Let's try a broad search of the CDA to see what information exists about cancers that were first diagnosed in the brain. To run this simple search, we would first construct a query in `Q` where the syntax is `Q('<name_of_column> = "<unique_term>"'` and save it to a variable `myquery`:

In [2]:
myquery = Q('primary_diagnosis_site = "brain"')


<div class="cdawarn" style="background-color:#f9cfbf;color:black;padding:20px;">
    
<h3>Where did those terms come from?</h3>
    
If you aren't sure how we knew what terms to put in our search, please refer back to the <a href="../SearchTerms">What search terms are available?</a> notebook. 
</div>

### subject
Now we can use that query to search any of information types. Let's start by looking at what subjects meet our criteria. To do that, we will send our query to the subject endpoint, then ask for it to run:

In [3]:
subjectresults = myquery.subject.run()

Getting results from database

Total execution time: 0
                        min 3.992 sec 3992 ms

We saved the output in a variable `subjectresults`, so we don't get much visible output. To see what our results are, we need to look into the variable. The simplest way is to call `subjectresults` directly:

In [4]:
subjectresults


            
            Offset: 0
            Count: 100
            Total Row Count: 3015
            More pages: True
            

This tells us four parameters that describe our results:

---

- **Offset:** This is how many rows of information we've told the query to skip in the data, here we didn't tell it to skip anything, so the offset is zero
- **Count:** This is how many rows the current page of our results table has. To keep searches fast, we default to pages with 100 rows.
- **Total Row Count:** This is how many rows are in the full results table
- **More pages:** This is always a True or False. False means that our current page has all the available results. True means that we will see only the first 100 results in this table, and will need to page through for more.

---

In any results variable, you can also use `print()` to display a 'pretty print' version of the results. This color codes the display in a way that can make it easier to find the information you care about:


In [5]:
print(subjectresults)

Offset: 0
            Count: 100
            Total Row Count: 3015
            More pages: True

We have 2384 subject matches, let's look at the actual table. The easiest way to do this is by using the python function `.to_dataframe()` on our `subjectresults` variable:

In [6]:
subjectresults.to_dataframe()

subject_id                                 subject_identifier  \
0    900-00-5445        [{'system': 'IDC', 'value': '900-00-5445'}]   
1         C16974             [{'system': 'PDC', 'value': 'C16974'}]   
2        C270477            [{'system': 'PDC', 'value': 'C270477'}]   
3         C30012             [{'system': 'PDC', 'value': 'C30012'}]   
4         C38868             [{'system': 'PDC', 'value': 'C38868'}]   
..           ...                                                ...   
95  TCGA-DH-A7UV  [{'system': 'GDC', 'value': 'TCGA-DH-A7UV'}, {...   
96  TCGA-DU-5851  [{'system': 'GDC', 'value': 'TCGA-DU-5851'}, {...   
97  TCGA-DU-A76R  [{'system': 'GDC', 'value': 'TCGA-DU-A76R'}, {...   
98  TCGA-DU-A7TA  [{'system': 'GDC', 'value': 'TCGA-DU-A7TA'}, {...   
99  TCGA-HT-7607  [{'system': 'GDC', 'value': 'TCGA-HT-7607'}, {...   

         species     sex   race               ethnicity  days_to_birth  \
0   homo sapiens    None   None                    None            NaN   
1   homo sapiens    male  white  not hispanic or latino            NaN   
2   homo sapiens    male  white  not hispanic or latino            NaN   
3   homo sapiens    male  white  not hispanic or latino            NaN   
4   homo sapiens  female  white  not hispanic or latino            NaN   
..           ...     ...    ...                     ...            ...   
95  homo sapiens    male  white  not hispanic or latino            NaN   
96  homo sapiens  female  white  not hispanic or latino            NaN   
97  homo sapiens    male  white  not hispanic or latino            NaN   
98  homo sapiens    male  white  not hispanic or latino            NaN   
99  homo sapiens  female  white  not hispanic or latino            NaN   

                           subject_associated_project vital_status  \
0                                         [rembrandt]         None   
1   [Proteogenomic Analysis of Pediatric Brain Can...        Alive   
2   [Proteogenomic Analysis of Pediatric Brain Can...        Alive   
3   [Proteogenomic Analysis of Pediatric Brain Can...         Dead   
4   [Proteogenomic Analysis of Pediatric Brain Can...        Alive   
..                                                ...          ...   
95                               [tcga_lgg, TCGA-LGG]        Alive   
96                               [tcga_lgg, TCGA-LGG]        Alive   
97                               [tcga_lgg, TCGA-LGG]         Dead   
98                               [tcga_lgg, TCGA-LGG]        Alive   
99                               [tcga_lgg, TCGA-LGG]         Dead   

    days_to_death cause_of_death  
0             NaN           None  
1             NaN   Not Reported  
2             NaN   Not Reported  
3             NaN   Not Reported  
4             NaN   Not Reported  
..            ...            ...  
95            NaN           None  
96            NaN           None  
97            NaN           None  
98            NaN           None  
99            NaN           None  

[100 rows x 11 columns]

By default `to_dataframe()` shows us the first and last five rows for the first page of our results, so we can easily preview our data.

Since we queried the Subject endpoint, our default results tell us Subject level information, that is, information about unique individuals: their sex, race, age, species, etc. The `id` column tells us the unique identifier for each individual. The identifier column has nested information about what study or studies a Subject participated in, and will list all of their researchsubject identifiers. 

The `to_dataframe()` function converts the results to a pandas dataframe. So if we save the dataframe to a variable, we can use any [pandas dataframe functions](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) to explore it. For example, lets see whether any of the Subjects in our first 100 results are black or african american. First we'll save the results to a dataframe, then subset that dataframe to only show rows where the word "black" appears in the "race" column. "NAs" which are shown as "None" in these tables, so for our filter to work, we'll need to specifically tell it to ignore NAs:


In [7]:
subjectdata = subjectresults.to_dataframe()
subjectdata[subjectdata['race'].str.contains("black", na=False)]

subject_id                                 subject_identifier  \
75  TCGA-DU-5872  [{'system': 'GDC', 'value': 'TCGA-DU-5872'}, {...   
81  TCGA-OX-A56R  [{'system': 'GDC', 'value': 'TCGA-OX-A56R'}, {...   

         species     sex                       race               ethnicity  \
75  homo sapiens  female  black or african american  not hispanic or latino   
81  homo sapiens    male  black or african american  not hispanic or latino   

    days_to_birth subject_associated_project vital_status  days_to_death  \
75            NaN       [tcga_lgg, TCGA-LGG]        Alive            NaN   
81            NaN       [TCGA-GBM, tcga_gbm]         Dead            NaN   

   cause_of_death  
75           None  
81           None

There are subjects in our first hundred results that meet the criteria. If we just want to be sure that the data contains some value, this might be good enough. But often we want to search the entire set of results and not just the first page. 

We'll cover how to work with large results dataframes in the <a href="../Pagination">Pagination</a> notebook. Or, learn how to get summary information from search results in the <a href="../DataSummaries">Data Summaries</a> notebook.

---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Subject Field Definitions</h3>

<i>A patient entity captures the study-independent metadata for research subjects. Human research subjects are usually not traceable to a particular person to protect the subjects privacy.</i>

    
<ul>
<li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system."</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
<li><b>species:</b> The taxonomic group (e.g. species) of the subject.</li>
<li><b>sex:</b> The biologic character or quality that distinguishes male and female from one another as expressed by analysis of the person's gonadal, morphologic (internal and external), chromosomal, and hormonal characteristics.</li>
<li><b>race:</b> An arbitrary classification of a taxonomic group that is a division of a species. It usually arises as a consequence of geographical isolation within a species and is characterized by shared heredity, physical attributes and behavior, and in the case of humans, by common history, nationality, or geographic distribution. The provided values are based on the categories defined by the U.S. Office of Management and Business and used by the U.S. Census Bureau.</li>
<li><b>ethnicity:</b> An individual's self-described social and cultural grouping, specifically whether an individual describes themselves as Hispanic or Latino. The provided values are based on the categories defined by the U.S. Office of Management and Business and used by the U.S. Census Bureau.</li>
<li><b>days_to_birth:</b> Number of days between the date used for index and the date from a person's date of birth represented as a calculated negative number of days.</li>
<li><b>subject_associated_project:</b> The list of Projects associated with the Subject.</li>
<li><b>vital_status:</b> Coded value indicating the state or condition of being living or deceased; also includes the case where the vital status is unknown.</li>
<li><b>days_to_death:</b> Number of days between the date used for index and the date from a person's date of death represented as a calculated number of days.</li>
<li><b>cause_of_death:</b> Coded value indicating the circumstance or condition that results in the death of the subject.</li>
</ul>  

</div>
    
---

### researchsubject

If we're interested in what researchsubjects meet our criteria, we can run our query against the researchsubject endpoint:

In [8]:
researchsubjectresults = myquery.researchsubject.run()
print(researchsubjectresults)

Getting results from database

Total execution time: 0
                        min 4.626 sec 4626 ms

Offset: 0
            Count: 100
            Total Row Count: 4347
            More pages: True

Now we see that our 2384 subjects have 3716 researchsubjects between them, that means that some, but not all, of our subjects were participants in more than one study. Let's peek at the data:

In [9]:
researchsubjectresults.to_dataframe()

researchsubject_id  \
0          d2b1569a-5129-485a-ad79-a63661818f78   
1             PGBM-010__brain_tumor_progression   
2          be68cfb0-c47f-49a1-9e39-b0b3e57372fc   
3   RIDER Neuro MRI-1863082306__rider_neuro_mri   
4             PGBM-011__brain_tumor_progression   
..                                          ...   
95                                   W4__ivygap   
96                                   W9__ivygap   
97                                   W6__ivygap   
98                                   W7__ivygap   
99                                  W33__ivygap   

                           researchsubject_identifier  \
0   [{'system': 'GDC', 'value': 'd2b1569a-5129-485...   
1   [{'system': 'IDC', 'value': 'PGBM-010__brain_t...   
2   [{'system': 'GDC', 'value': 'be68cfb0-c47f-49a...   
3   [{'system': 'IDC', 'value': 'RIDER Neuro MRI-1...   
4   [{'system': 'IDC', 'value': 'PGBM-011__brain_t...   
..                                                ...   
95         [{'system': 'IDC', 'value': 'W4__ivygap'}]   
96         [{'system': 'IDC', 'value': 'W9__ivygap'}]   
97         [{'system': 'IDC', 'value': 'W6__ivygap'}]   
98         [{'system': 'IDC', 'value': 'W7__ivygap'}]   
99        [{'system': 'IDC', 'value': 'W33__ivygap'}]   

   member_of_research_project primary_diagnosis_condition  \
0   EXCEPTIONAL_RESPONDERS-ER                     Gliomas   
1     brain_tumor_progression                        None   
2                   GENIE-JHU              Neoplasms, NOS   
3             rider_neuro_mri                        None   
4     brain_tumor_progression                        None   
..                        ...                         ...   
95                     ivygap                        None   
96                     ivygap                        None   
97                     ivygap                        None   
98                     ivygap                        None   
99                     ivygap                        None   

   primary_diagnosis_site                  subject_id  
0                   Brain                     ER-ABXP  
1                   Brain                    PGBM-010  
2                   Brain             GENIE-JHU-02615  
3                   Brain  RIDER Neuro MRI-1863082306  
4                   Brain                    PGBM-011  
..                    ...                         ...  
95                  Brain                          W4  
96                  Brain                          W9  
97                  Brain                          W6  
98                  Brain                          W7  
99                  Brain                         W33  

[100 rows x 6 columns]

Each row from the researchsubject endpoint results tells us about a subject in a given study. Using this endpoint we can find out information like what studies fit our search criteria, and also get data that we can filter to have only subjects from multiple studies, or only subjects from single studies.

Any given subject will have one row per study they participated in. The subject_id in the last column of this view is the same as the `id` in the first column of the Subjects endpoint results. You can use this to combine information across endpoints, which is covered near the end of the <a href="../BuildingACohort/#merging-results-across-endpoints">Cohort Building workflow</a> notebook.

---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>ResearchSubject Field Definitions</h3>

<i>A research subject is the entity of interest in a specific research study or project, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subjects privacy. This entity plays the role of the case_id in existing data. A subject who participates in 3 studies will have 3 researchsubject IDs</i>
    
<ul>
<li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system. For CDA, this is case_id.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
<li><b>member_of_research_project:</b> A reference to the Project(s) of which this ResearchSubject is a member.</li>
<li><b>primary_diagnosis_condition:</b> The text term used to describe the type of malignant disease, as categorized by the World Health Organization's (WHO) <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> (ICD-O). This attribute represents the disease that qualified the subject for inclusion on the ResearchProject.</li>
<li><b>primary_diagnosis_site:</b> The text term used to describe the primary site of disease, as categorized by the World Health Organization's (WHO) <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> (ICD-O). This categorization groups cases into general categories. This attribute represents the primary site of disease that qualified the subject for inclusion on the ResearchProject.</li>
<li><b>subject_id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system. Can be joined to the `id` field from subject results</li>
</ul>  

</div>
    
---

### diagnosis

The diagnosis endpoint is an extension of the researchsubject endpoint, and returns information about researchsubjects that have a diagnosis that meets our search criteria:

In [10]:
diagnosisresults = myquery.diagnosis.run()
diagnosisresults.to_dataframe()

Getting results from database

Total execution time: 0
                        min 4.307 sec 4307 ms

diagnosis_id  \
0   36c60545-fef6-59e6-9d56-95e2d07daae0   
1   4283063f-fad7-561f-913e-9778439af9db   
2   53846869-2114-5ff2-9728-15f5f22fa72a   
3   78cacf8c-d55b-585d-b592-d28b64223411   
4   8c8550bf-0371-5742-a5e2-c48bd6070bf1   
..                                   ...   
95  056a6557-61a3-55ba-bc00-bc56bc39502e   
96  0612da88-f21f-4eb8-83db-471cff65d4f8   
97  0eb38b23-d59a-40a5-9318-e1b3a3d92d1b   
98  16201f1d-5dfc-5b22-bddb-0f227114c078   
99  20096542-0b59-55ca-b05d-e0b5d91af39a   

                                 diagnosis_identifier       primary_diagnosis  \
0   [{'system': 'GDC', 'value': '36c60545-fef6-59e...            Glioblastoma   
1   [{'system': 'GDC', 'value': '4283063f-fad7-561...        Astrocytoma, NOS   
2   [{'system': 'GDC', 'value': '53846869-2114-5ff...            Glioblastoma   
3   [{'system': 'GDC', 'value': '78cacf8c-d55b-585...            Glioblastoma   
4   [{'system': 'GDC', 'value': '8c8550bf-0371-574...            Glioblastoma   
..                                                ...                     ...   
95  [{'system': 'GDC', 'value': '056a6557-61a3-55b...            Glioblastoma   
96  [{'system': 'GDC', 'value': '0612da88-f21f-4eb...            Glioblastoma   
97  [{'system': 'GDC', 'value': '0eb38b23-d59a-40a...            Glioblastoma   
98  [{'system': 'GDC', 'value': '16201f1d-5dfc-5b2...            Glioblastoma   
99  [{'system': 'GDC', 'value': '20096542-0b59-55c...  Oligodendroglioma, NOS   

    age_at_diagnosis morphology stage         grade method_of_diagnosis  \
0            15660.0     9440/3  None  not reported                None   
1            11233.0     9400/3  None  not reported                None   
2            25684.0     9440/3  None  not reported                None   
3            28583.0     9440/3  None  not reported                None   
4            11668.0     9440/3  None  not reported                None   
..               ...        ...   ...           ...                 ...   
95           17402.0     9440/3  None  not reported                None   
96           17506.0     9440/3  None  Not Reported                None   
97           27364.0     9440/3  None  Not Reported                None   
98           21069.0     9440/3  None  not reported                None   
99           19991.0     9450/3  None  not reported                None   

      subject_id                    researchsubject_id  
0   TCGA-02-0102  f3e57166-c428-4a44-9396-679e3a2b16a2  
1   TCGA-HT-7606  ffcec8e5-9fd3-4b42-a7cb-74761f713cf4  
2   TCGA-14-1825  e0e1b5b3-6e3c-4b79-aa2a-32b320c3e45a  
3   TCGA-12-0615  141a1ef1-9be6-46d1-b445-305b222727d2  
4   TCGA-02-0069  73b1136f-c4b6-472a-b21d-2c789bd53d26  
..           ...                                   ...  
95  TCGA-02-0025  6630e547-2596-4fb7-a657-c19497ff13b2  
96     C3N-02782  c1314846-7acd-4d94-a073-0ffe5cddb595  
97     C3N-01367  4a21fc0b-d84d-4bc3-be3b-b2c0b14b97be  
98  TCGA-12-0620  1dbfbff7-d048-4770-ae02-e6a287256210  
99  TCGA-VM-A8CA  cdd1fd4c-3427-496c-8391-4aeb6730d3cc  

[100 rows x 10 columns]


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Diagnosis Field Definitions</h3>

<i>A collection of characteristics that describe an abnormal condition of the body as assessed at a point in time. May be used to capture information about neoplastic and non-neoplastic conditions.</i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>primary_diagnosis:</b> The diagnosis instance that qualified a subject for inclusion on a ResearchProject.</li>
  <li><b>age_at_diagnosis:</b> The age in days of the individual at the time of diagnosis.</li>
  <li><b>morphology:</b> Code that represents the histology of the disease using the third edition of the <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a>, published in 2000, used principally in tumor and cancer registries for coding the site (topography) and the histology (morphology) of neoplasms.</li>
  <li><b>stage:</b> The extent of a cancer in the body. Staging is usually based on the size of the tumor, whether lymph nodes contain cancer, and whether the cancer has spread from the original site to other parts of the body. Different diseases may use different staging criteria, please refer to the originating data source to see what staging system is reported</li>
  <li><b>grade:</b> The degree of abnormality of cancer cells, a measure of differentiation, the extent to which cancer cells are similar in appearance and function to healthy cells of the same tissue type. The degree of differentiation often relates to the clinical behavior of the particular tumor. Based on the microscopic findings, tumor grade is commonly described by one of four degrees of severity. Histopathologic grade of a tumor may be used to plan treatment and estimate the future course, outcome, and overall prognosis of disease. Certain types of cancers, such as soft tissue sarcoma, primary brain tumors, lymphomas, and breast have special grading systems.</li>
  <li><b>method_of_diagnosis:</b> The method used to confirm the patients malignant diagnosis.</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---




### treatment

The treatment endpoint is an extension of diagnosis and returns information about treatments undertaken on researchsubjects that have a given diagnosis that meets our search criteria:

In [11]:
treatmentresults = myquery.treatment.run()
treatmentresults.to_dataframe()

Getting results from database

Total execution time: 0
                        min 4.089 sec 4089 ms

treatment_id  \
0   1d33518c-20cf-5dbd-8277-88f9c43734ac   
1   2884ce00-2e57-5e8b-8878-d7059195de38   
2   335d0fe9-8775-553a-9689-4f624d530a8b   
3   4d6b7c47-75f7-5c97-b565-a6d9142e3e88   
4   611741c0-c4b6-5900-bd5f-30ae047564f3   
..                                   ...   
95  1b6265b9-4b00-54de-ae60-2f1a3845da23   
96  1d8867db-17a1-5553-8e50-0acfdbb13e33   
97  26929d96-dd87-5a21-8d09-3ca0e704b52d   
98  351cde63-8677-53d4-9531-69e50addb157   
99  4439da8a-f1a6-50fd-8a08-51ab4dd74613   

                                 treatment_identifier treatment_type  \
0   [{'system': 'GDC', 'value': '1d33518c-20cf-5db...           None   
1   [{'system': 'GDC', 'value': '2884ce00-2e57-5e8...           None   
2   [{'system': 'GDC', 'value': '335d0fe9-8775-553...           None   
3   [{'system': 'GDC', 'value': '4d6b7c47-75f7-5c9...           None   
4   [{'system': 'GDC', 'value': '611741c0-c4b6-590...           None   
..                                                ...            ...   
95  [{'system': 'GDC', 'value': '1b6265b9-4b00-54d...           None   
96  [{'system': 'GDC', 'value': '1d8867db-17a1-555...           None   
97  [{'system': 'GDC', 'value': '26929d96-dd87-5a2...           None   
98  [{'system': 'GDC', 'value': '351cde63-8677-53d...           None   
99  [{'system': 'GDC', 'value': '4439da8a-f1a6-50f...           None   

   treatment_outcome days_to_treatment_start days_to_treatment_end  \
0               None                    None                  None   
1               None                    None                  None   
2               None                    None                  None   
3               None                    None                  None   
4               None                    None                  None   
..               ...                     ...                   ...   
95              None                    None                  None   
96              None                    None                  None   
97              None                    None                  None   
98              None                    None                  None   
99              None                    None                  None   

   therapeutic_agent treatment_anatomic_site treatment_effect  \
0               None                    None             None   
1               None                    None             None   
2               None                    None             None   
3               None                    None             None   
4               None                    None             None   
..               ...                     ...              ...   
95              None                    None             None   
96              None                    None             None   
97              None                    None             None   
98              None                    None             None   
99              None                    None             None   

   treatment_end_reason number_of_cycles    subject_id  \
0                  None             None  TCGA-S9-A7IX   
1                  None             None  TCGA-32-5222   
2                  None             None  TCGA-06-6698   
3                  None             None  TCGA-12-1095   
4                  None             None  TCGA-14-4157   
..                  ...              ...           ...   
95                 None             None  TCGA-S9-A7QX   
96                 None             None  TCGA-06-0141   
97                 None             None  TCGA-06-5412   
98                 None             None  TCGA-CS-4943   
99                 None             None  TCGA-02-2486   

                      researchsubject_id                          diagnosis_id  
0   a80b0fab-6189-497f-997f-77234b69179c  7a822cf0-7339-5657-9cae-342a5db4fa1a  
1   e876dd29-68b0-4bf1-83d1-488c40068a35  ac0b1caf-54a0-5d33-8eba-acfacf485f32  
2   c129c34a-8d40-4d98-b9ad-a10d3f7b3ee0  6939f776-7ae7-5e1f-83f2-1a178c


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Treatment Field Definitions</h3>

<i><i> Medication administration or other treatment types. A single research subject may have multiple treatments for a single diagnosis, and/or different diagnoses, and different treatments, across different studies</i></i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>treatment_type:</b> The treatment type including medication/therapeutics or other procedures.</li>
  <li><b>treatment_outcome:</b> The final outcome of the treatment.</li>
  <li><b>days_to_treatment_start:</b> The timepoint at which the treatment started.</li>
  <li><b>days_to_treatment_end:</b>The timepoint at which the treatment ended. </li>
  <li><b>therapeutic_agent:</b> One or more therapeutic agents as part of this treatment.</li>
  <li><b>treatment_anatomic_site:</b> The anatomical site that the treatment targets.</li>
  <li><b>treatment_effect:</b>The effect of a treatment on the diagnosis or tumor. </li>
  <li><b>treatment_end_reason:</b>The reason the treatment ended. </li>
  <li><b>number_of_cycles:</b>The number of treatment cycles the subject received. </li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
  <li><b>researchsubject_diagnosis_id:</b> An identifier for the diagnosis. Can be joined to the `id` field from diagnosis results</li>
</ul>  

</div>
    
---




### specimen

We can use this same query to see what specimens are available for brain tissue at the CDA:

In [12]:
specimenresults =  myquery.specimen.run()
specimenresults

Getting results from database

Total execution time: 0
                        min 7.675 sec 7675 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 39220
            More pages: True
            

Nearly 40,000 specimens meet our search criteria! We would typically expect this number to be much larger than our number of subjects or researchsubjects. First because studies will often take more than one sample per subject, and second because any given specimen might be aliquoted out to be used in multiple tests. Since we didn't specify any further filters, our results will return all of these as separate specimens. Let's look at a few:

In [13]:
specimenresults.to_dataframe()

specimen_id  \
0   000d919d-9a56-4cea-a46c-f0854b2ccecc   
1   00150f31-f517-425e-a9f9-89e3f3c53a12   
2   00580e36-eea0-455e-90e7-f5ee7bd1cd7b   
3   006f748a-3085-43e1-9738-671f523f3bc7   
4   00bb2e94-ddd4-417c-a18d-e038640a5c4d   
..                                   ...   
95  3d477ac1-3bdf-4410-9594-1a3a1412c6ae   
96  3e331ee6-161c-4a6c-8a58-afe66d0b5970   
97  3e8fb17d-dea6-4a33-9581-2a8b5f2a7e14   
98  3ee0e7e0-b88d-469a-b75b-09c16775de4b   
99  40ccc704-6fcc-4e7a-8b73-8106a0445349   

                                  specimen_identifier  \
0   [{'system': 'GDC', 'value': '000d919d-9a56-4ce...   
1   [{'system': 'GDC', 'value': '00150f31-f517-425...   
2   [{'system': 'GDC', 'value': '00580e36-eea0-455...   
3   [{'system': 'GDC', 'value': '006f748a-3085-43e...   
4   [{'system': 'GDC', 'value': '00bb2e94-ddd4-417...   
..                                                ...   
95  [{'system': 'GDC', 'value': '3d477ac1-3bdf-441...   
96  [{'system': 'GDC', 'value': '3e331ee6-161c-4a6...   
97  [{'system': 'GDC', 'value': '3e8fb17d-dea6-4a3...   
98  [{'system': 'GDC', 'value': '3ee0e7e0-b88d-469...   
99  [{'system': 'GDC', 'value': '40ccc704-6fcc-4e7...   

   specimen_associated_project  days_to_collection primary_disease_type  \
0                     TCGA-GBM                 NaN              Gliomas   
1                     TCGA-LGG              1453.0              Gliomas   
2                     TCGA-LGG                 NaN              Gliomas   
3                     TCGA-LGG                 NaN              Gliomas   
4                      CPTAC-3                 NaN              Gliomas   
..                         ...                 ...                  ...   
95                    TCGA-LGG                 NaN              Gliomas   
96                    TCGA-GBM                 NaN              Gliomas   
97                    TCGA-LGG                 NaN              Gliomas   
98                    TCGA-LGG                 NaN              Gliomas   
99                    TCGA-LGG               627.0              Gliomas   

   anatomical_site  source_material_type specimen_type  \
0             None  Blood Derived Normal       aliquot   
1             None         Primary Tumor       analyte   
2             None         Primary Tumor       aliquot   
3             None  Blood Derived Normal       portion   
4             None  Blood Derived Normal        sample   
..             ...                   ...           ...   
95            None         Primary Tumor       aliquot   
96            None         Primary Tumor       aliquot   
97            None         Primary Tumor       aliquot   
98            None  Blood Derived Normal       aliquot   
99            None         Primary Tumor       aliquot   

                   derived_from_specimen derived_from_subject    subject_id  \
0   108ad5dc-0ee2-4ec0-a7f1-bd0ed0ae9142         TCGA-12-0616  TCGA-12-0616   
1   68cfd499-8ca0-4898-9f31-8ba2bfb9831e         TCGA-TM-A7CF  TCGA-TM-A7CF   
2   fe4709d2-fab8-4011-97d9-bd74e86f79d3         TCGA-FG-6689  TCGA-FG-6689   
3   7e22ffc3-818b-46e9-abf2-5fd0db49efde         TCGA-HT-7482  TCGA-HT-7482   
4                       initial specimen            C3N-01369     C3N-01369   
..                                   ...                  ...           ...   
95  eaf9c29f-f2e6-4362-9464-443b8605ea22         TCGA-CS-5395  TCGA-CS-5395   
96  12374c34-7d21-4def-b02a-ca1e20aa4e89         TCGA-06-0197  TCGA-06-0197   
97  005f8700-04c7-48ad-a22c-976e0a8a712e         TCGA-DU-7006  TCGA-DU-7006   
98  cedc3806-9f41-4590-9df7-65e2f4414d9f         TCGA-HT-7690  TCGA-HT-7690   
99  34f84bc6-0d7b-45aa-9fee-dbd1cda8dfb0         TCGA-TM-A84H  TCGA-TM-A84H   

                      researchsubject_id  
0   edcfac32-adb4-4fe7-86a0-309d3c6430e5  
1   033c1f48-9225-4159-b23b-7042272877da  
2   8d07b1e8-5d1d-4a43-b616-d29dd58ce5fe  
3   448d3e3b-622c-4e79-bb1d-b5f2e92970e8  
4   e59a7d58-adf4-4d05-8a7f-c3013285b568  
..        


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Specimen Field Definitions</h3>

<i><i>Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</i>
 A given specimen will have only a single subject ID and a single researchsubject ID</i>
    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>associated_project:</b> The Project associated with the specimen.</li>
  <li><b>days_to_collection:</b> The number of days from the index date to either the date a sample was collected for a specific study or project, or the date a patient underwent a procedure (e.g. surgical resection) yielding a sample that was eventually used for research.</li>
  <li><b>primary_disease_type:</b> The text term used to describe the type of malignant disease, as categorized by the World Health Organization's (WHO) International Classification of Diseases for Oncology (ICD-O). This attribute represents the disease that qualified the subject for inclusion on the ResearchProject.</li>
  <li><b>anatomical_site:</b> Per GDC Dictionary, the text term that represents the name of the primary disease site of the submitted tumor sample; recommend dropping tumor; biospecimen_anatomic_site.</li>
  <li><b>source_material_type:</b> The general kind of material from which the specimen was derived, indicating the physical nature of the source material.</li>
  <li><b>specimen_type:</b> The high-level type of the specimen, based on its how it has been derived from the original extracted sample. One of: analyte, aliquot, portion, sample, or slide.</li>
  <li><b>derived_from_specimen:</b> A source/parent specimen from which this one was directly derived.</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---


### file

The file endpoint returns information about files that meet our search criteria, and that belong to subjects, researchsubjects, or specimens. You can think of three of these endpoints as being nested: 

`subject.file` is the broadest search. It will give you all files that have the parameters of your query
`researchsubject.file` will give you all the files that match your query, and that are attached to specific studies and/or specimens. It will not return subject level files.
`specimen.file` will return all the files directly attached to specimen, but not those attached to the researchsubject or subject those specimens came from.


In [14]:
myquery.subject.file.run()

Getting results from database

Total execution time: 0
                        min 15.846 sec 15846 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 4969686
            More pages: True
            

In [15]:
myquery.researchsubject.file.run()

Getting results from database

Total execution time: 0
                        min 18.895 sec 18895 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 4969666
            More pages: True
            

In [16]:
myquery.specimen.file.run()

Getting results from database

Total execution time: 0
                        min 10.088 sec 10088 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 771260
            More pages: True
            

As you might expect, searching file gives us a huge number of results. This is great if you are surveying what kind of data is available, but is less useful for getting a coherent cohort. 

A better way to get files for a specific cohort is to build a more complex query using <a href="../Operators">Operators</a>. 

Another useful way to look at high level information is to use our counts feature which returns summary information rather than the full search results. Check out the <a href="../DataSummaries">Data Summaries tutorial</a> to try it.



---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>File Field Definitions</h3>

<i>A file is an information-bearing electronic object that contains a physical embodiment of some information using a particular character encoding.</i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>label:</b> Short name or abbreviation for dataset. Maps to rdfs:label.</li>
  <li><b>data_catagory:</b> Broad categorization of the contents of the data file.</li>
  <li><b>data_type:</b> Specific content type of the data file.</li>
  <li><b>file_format:</b> Format of the data files.</li>
  <li><b>associated_project:</b> A reference to the Project(s) of which this ResearchSubject is a member. The associated_project may be embedded using the ref definition or may be a reference to the id for the Project - or a URI expressed as a string to an existing entity.</li>
  <li><b>drs_uri:</b> A string of characters used to identify a resource on the Data Repo Service(DRS). Can be used to retrieve this specific file from a server.</li>
  <li><b>byte_size:</b> Size of the file in bytes. Maps to dcat:byteSize.</li>
  <li><b>checksum:</b> The md5 value for the file. A digit representing the sum of the correct digits in a piece of stored or transmitted digital data, against which later comparisons can be made to detect errors in the data.</li>
  <li><b>data_modality:</b> Data modality describes the biological nature of the information gathered as the result of an activity, independent of the technology or methods used to produce the information. Always one of "Genomic", "Proteomic", or "Imaging".</li>
  <li><b>imaging_modality:</b> An imaging modality describes the imaging equipment and/or method used to acquire certain structural or functional information about the body. These include but are not limited to computed tomography (CT) and magnetic resonance imaging (MRI). Taken from the DICOM standard.</li>
  <li><b>dbgap_accession_number:</b> The dbgap accession number for the project.</li>
</ul>  

</div>
    
---

### mutation

The file endpoint returns molecular data about specific mutations that meet our search criteria. Search is currently limited to the TCGA-READ project from GDC:

In [17]:
myquery.mutation.run()

Getting results from database

Total execution time: 0
                        min 10.41 sec 10410 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 904
            More pages: True
            

In [18]:
mutationresults = myquery.mutation.run()
mutationresults.to_dataframe()

Getting results from database

Total execution time: 0
                        min 4.279 sec 4279 ms

project_short_name  case_barcode sample_barcode_tumor  \
0            TCGA-GBM  TCGA-12-0691     TCGA-12-0691-01A   
1            TCGA-LGG  TCGA-DU-5849     TCGA-DU-5849-01A   
2            TCGA-LGG  TCGA-QH-A6CX     TCGA-QH-A6CX-01A   
3            TCGA-LGG  TCGA-QH-A6X9     TCGA-QH-A6X9-01A   
4            TCGA-LGG  TCGA-S9-A6WP     TCGA-S9-A6WP-01A   
..                ...           ...                  ...   
95           TCGA-LGG  TCGA-S9-A7QZ     TCGA-S9-A7QZ-01A   
96           TCGA-LGG  TCGA-TQ-A7RS     TCGA-TQ-A7RS-01A   
97           TCGA-GBM  TCGA-06-0167     TCGA-06-0167-01A   
98           TCGA-GBM  TCGA-06-0749     TCGA-06-0749-01A   
99           TCGA-GBM  TCGA-14-1453     TCGA-14-1453-01A   

   sample_barcode_normal         aliquot_barcode_tumor  \
0       TCGA-12-0691-10A  TCGA-12-0691-01A-01W-0348-08   
1       TCGA-DU-5849-10A  TCGA-DU-5849-01A-11D-1705-08   
2       TCGA-QH-A6CX-10A  TCGA-QH-A6CX-01A-11D-A32B-08   
3       TCGA-QH-A6X9-10B  TCGA-QH-A6X9-01A-12D-A32B-08   
4       TCGA-S9-A6WP-10A  TCGA-S9-A6WP-01A-12D-A34A-08   
..                   ...                           ...   
95      TCGA-S9-A7QZ-10A  TCGA-S9-A7QZ-01A-12D-A34J-08   
96      TCGA-TQ-A7RS-10A  TCGA-TQ-A7RS-01A-12D-A33T-08   
97      TCGA-06-0167-10A  TCGA-06-0167-01A-01D-1491-08   
98      TCGA-06-0749-10A  TCGA-06-0749-01A-01D-1492-08   
99      TCGA-14-1453-10A  TCGA-14-1453-01A-01W-0611-08   

          aliquot_barcode_normal Hugo_Symbol  Entrez_Gene_Id Center  \
0   TCGA-12-0691-10A-01W-0348-08        AKNA           80709     BI   
1   TCGA-DU-5849-10A-01D-1705-08      CLEC9A          283420     BI   
2   TCGA-QH-A6CX-10A-01D-A329-08      MSL3P1          151507     BI   
3   TCGA-QH-A6X9-10B-01D-A329-08      CLSTN3            9746     BI   
4   TCGA-S9-A6WP-10A-01D-A34A-08      PABPN1            8106     BI   
..                           ...         ...             ...    ...   
95  TCGA-S9-A7QZ-10A-01D-A34M-08       DIRC2           84925     BI   
96  TCGA-TQ-A7RS-10A-01D-A33W-08       WDR37           22884     BI   
97  TCGA-06-0167-10A-01D-1491-08     FAM86DP          692099     BI   
98  TCGA-06-0749-10A-01D-1492-08       RAMP3           10268     BI   
99  TCGA-14-1453-10A-01W-0612-08        VMP1           81671     BI   

   NCBI_Build  ...                        tumor_bam_uuid  \
0      GRCh38  ...  6b3ed94d-9fcb-414e-84ae-858ebaf27afe   
1      GRCh38  ...  17cd3685-a9f0-4c3e-b2e9-6ac133c94bbc   
2      GRCh38  ...  4b92d740-80f7-4e82-866e-0fcb8f256f56   
3      GRCh38  ...  23c33149-0739-44c5-80ac-4dc3d21946d4   
4      GRCh38  ...  e4e3b37e-97f8-4b0b-8b38-4ab2a149bb81   
..        ...  ...                                   ...   
95     GRCh38  ...  8d85c50d-05a0-4e4e-bcc8-cecd3eec968b   
96     GRCh38  ...  06350102-0c44-4846-935d-515a5b0da989   
97     GRCh38  ...  8c81d785-3895-4d7c-8394-a4b59dc0f104   
98     GRCh38  ...  a514f288-cdac-487f-ab7f-29a705889809   
99     GRCh38  ...  942eb146-7320-4d8b-ab06-435fdde84e55   

                         normal_bam_uuid  \
0   dae1068f-9ebb-4296-9288-e41e7a73de53   
1   7f6b4c30-1a7b-4616-9cac-223cc25d0b38   
2   3a5369d9-2a1f-463c-98dc-46e0e35f989a   
3   1f773ee3-4fbb-4801-808d-9798280c3082   
4   aac5c478-45f3-45c5-a4ba-d85aaaf85184   
..                                   ...   
95  c0b1c360-cd78-4d8d-907e-289578958a98   
96  3ba871b3-2d7f-4266-922b-627785c9e2e5   
97  32c41176-771e-4adb-ad32-459d7a1af233   
98  4aa7ba5e-fff3-4013-a0b5-523f463b4a3a   
99  6709d189-8d8e-428f-b637-0075f1d4e4ea   

                                 case_id GDC_FILTER       COSMIC MC3_Overlap  \
0   bb0aca6b-55ab-4b24-8e64-29e0c96a9862   wga_pair  COSM2153974        True   
1   5af8ada5-86b5-4832-8a8e-e975ecca5515       None         None       False   
2   cf29547a-305b-4e6e-8342-36aac0850065       None         None        True   
3   1ca2e1b3-e3eb-4740-a69a-800e3eb60787       None         None        True   
4   081b6532-0ed4-445a-becb-cb2290f4854d    gdc_pon         None        True   
..